# Marking of data lithofacies(对数据岩相进行标记)

## Import third-party packages(导入第三方包)

In [1]:
import os
import pandas as pd
import numpy as np

import math
import random
from pandas import set_option

In [2]:
# # calculate RMSE
# from sklearn.metrics import mean_squared_error, confusion_matrix
# import matplotlib.colors as colors
# # sklearn.metrics.mean_squared_error(y_true, y_pred, sample_weight=None, multioutput='uniform_average', squared=True)
# # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error

In [3]:
# print(tf.__version__)

numpy+mkl 版本为17.2  
tensorboard，tensorflow 版本为2.1.0  
pydot版本为1.4.1, graphviz 版本为0.13.2

## Import our custom package(导入自己的包)

In [4]:
# import senutil as sen
# # from rbflayer import RBFLayer, InitCentersRandom
# import senmodels_classification as smsc

In [5]:
# mpl.rcParams['font.sans-serif'] = ['SimHei']
# plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
# plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
set_option("display.max_rows", 15)
set_option('display.width', 200)
np.set_printoptions(suppress=True, threshold=5000)

In [6]:
# 使用GPU训练时候，打开下面注释
os.environ['CUDA_VISIBLE_DEVICES']='0'

## Preparation work (data processing, result saving location definition)准备工作

### Input Curve Dataset Position(输入曲线数据集位置)

In [7]:
usual_logs_DataPath = 'facies_data/usual_logs/'

In [8]:
filename_usual_logs = 'GY1_R_0.125m_整段_2350m-2746m_nolabel.csv'

usual_logs_DataPath = os.path.join(usual_logs_DataPath,filename_usual_logs)
print(usual_logs_DataPath)

facies_data/usual_logs/古页1_R_0.1m_整段_2350m-2746m_nolabel.csv


### Location of Petrographic Dataset（岩相数据集所在位置）

In [9]:
facies_logs_DataPath = 'facies_data/facies_logs/'

In [10]:
filename_facies_logs =  'GY1-lithofacies-0512.csv'

In [11]:
facies_logs_DataPath = os.path.join(facies_logs_DataPath,filename_facies_logs)

## Petrographic data calibration（岩相数据标定）

In [12]:
# 高分辨率相对于低分辨率的倍数, default value = 10 (对应于0.1m); 8 (对应于0.125m)
# A sampling rate of 1m
resolution = 10

### Read input curves and expert lithofacies classification data（读取输入曲线和专家岩相分类数据）

In [13]:
input_vector_data = pd.read_csv(usual_logs_DataPath,engine='python',encoding='GBK')
facies_vector = pd.read_csv(facies_logs_DataPath,engine='python',encoding='GBK')

In [14]:
input_vector_data

,DEPTH,GR,CNL,DT,DEN,MSFL,RS,RD
0,2350.0858,113.4619,15.9331,81.0,2.5902,11.5818,6.9659,7.4271
1,2350.1858,116.5655,16.5907,82.8,2.6165,7.7921,7.0260,7.5060
2,2350.2858,120.4164,17.4862,85.6,2.6315,6.2955,6.8697,7.3354
3,2350.3858,124.4936,18.4462,88.6,2.6291,6.0488,6.5242,6.9516
4,2350.4858,128.6494,19.1042,91.0,2.6026,5.9448,6.1002,6.5036
...,...,...,...,...,...,...,...,...
3956,2745.6858,89.4620,7.3066,71.0,2.5733,7.0714,12.5438,13.1089
3957,2745.7858,95.4042,7.5931,71.0,2.5846,6.9656,12.6673,13.2187
3958,2745.8858,104.3638,8.0731,71.0,2.5926,6.8015,12.8980,13.4263
3959,2745.9858,113.7063,8.9307,71.0,2.5983,6.5375,13.2500,13.7623


In [15]:
facies_vector

,层名,起始深度,终止深度,井号
0,低有机质纹层状页岩相,2304.46,2324.88,1
1,低有机质块状粉砂岩相,2324.88,2326.29,1
2,低有机质纹层状页岩相,2326.29,2336.86,1
3,低有机质块状粉砂岩相,2336.86,2337.16,1
4,低有机质纹层状页岩相,2337.16,2367.19,1
...,...,...,...,...
156,高有机质纹层状页岩相,2579.22,2583.54,1
157,低有机质块状介壳灰岩相,2583.54,2584.02,1
158,中有机质纹层状页岩相,2584.02,2584.42,1
159,高有机质纹层状页岩相,2584.42,2585.02,1


In [16]:
# 针对输入曲线
depth_cols = 'DEPTH'
# 针对岩相曲线
depth_begin_col = '起始深度'
depth_end_col = '终止深度'
facies_col = '层名'
well_name_col = '井号'

In [17]:
pd_well_no = np.full(len(input_vector_data),facies_vector[well_name_col][0])
pd_well_no

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### Lithofacies classification calibration（岩相类别标定）

In [18]:
facies_label = np.unique(facies_vector[facies_col])

'''
facies_label = [
    "High organic matter layered shale lithofacies",
    "High organic striated layered shale lithofacies",
    "Medium organic striated layered shale lithofacies",
    "Low organic matter striated layered shale lithofacies",
    "Medium to low organic matter massive dolomite lithofacies",
    "Low organic matter massive shell limestone lithofacies"
    "Low organic matter massive siltstone lithofacies"
]
'''

facies_labels = ['高有机质层状页岩相', '高有机质纹层状页岩相','中有机质纹层状页岩相','低有机质纹层状页岩相',
                '中低有机质块状白云岩相', '低有机质块状介壳灰岩相', '低有机质块状粉砂岩相']


# facies_labels = ['层状粘土质页岩', '纹层状粘土质页岩','层状长英质页岩','纹层状长英质页岩','介壳灰岩']

In [19]:
depth_min = 2350
depth_max = 2600
math.floor(depth_min),math.ceil(depth_max)

(2350, 2600)

In [20]:
# for i in range(math.floor(depth_min),math.ceil(depth_max)+1):
#     print(i)
len(facies_labels[0])

9

In [21]:
label_max_lenth = 0

In [22]:
for i in range(len(facies_labels)):
    if label_max_lenth < len(facies_labels[i]):
        label_max_lenth = len(facies_labels[i])

In [23]:
len(input_vector_data)

3961

In [24]:
def generate_random_str(randomlength):
    """
    生成一个指定长度的随机字符串
    """
    random_str = ''
    base_str = 'ABCDEFGHIGKLMNOPQRSTUVWXYZabcdefghigklmnopqrstuvwxyz0123456789'
    length = len(base_str) - 1
    for i in range(randomlength):
        random_str += base_str[random.randint(0, length)]
    return random_str
# ————————————————
# 版权声明：本文为CSDN博主「geerniya」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
# 原文链接：https://blog.csdn.net/geerniya/java/article/details/78497765

In [25]:
fill_label = generate_random_str(label_max_lenth)
fill_label

'wz8BrHlwNb4'

In [26]:
facies_list = np.full(len(input_vector_data),"no_label")

In [27]:
facies_list = np.full(len(input_vector_data),"no_label")

lower_depth_index = 0
# 深度列标尺初始化
# current_depth = input_vector_data[depth_cols][0]
for i in range(len(facies_vector)):
    depth_min = facies_vector[depth_begin_col][i]
    depth_max = facies_vector[depth_end_col][i]
    current_facies_label = facies_vector[facies_col][i]
    current_facies_label = current_facies_label.strip()
    # 获取当前岩相表的岩性id
    current_facies_id = facies_labels.index(current_facies_label)
    # current_well_no = facies_vector[well_name_col][i]
    if(i%10==0):
        print("depth_min,depth_max:",depth_min,depth_max)
        print("No_" + str(i) + "_layers")
    # lower_depth = math.floor(depth_min)
    # upper_depth = math.ceil(depth_max)
    for j in range(0,len(input_vector_data)):
        current_depth = input_vector_data[depth_cols][j]
        if depth_min <= current_depth:
            if current_depth <= depth_max:
                facies_list[j] = current_facies_id 
                # pd_well_no[j]  = current_well_no
                lower_depth_index = j
                # print(str(j) + ":",current_facies_id)
print("Label Work Finished!")

depth_min,depth_max: 2304.46 2324.88
No_0_layers
depth_min,depth_max: 2385.35 2387.4
No_10_layers
depth_min,depth_max: 2416.62 2417.45
No_20_layers
depth_min,depth_max: 2425.86 2445.17
No_30_layers
depth_min,depth_max: 2456.34 2458.51
No_40_layers
depth_min,depth_max: 2462.46 2462.64
No_50_layers
depth_min,depth_max: 2477.51 2487.88
No_60_layers
depth_min,depth_max: 2493.21 2495.04
No_70_layers
depth_min,depth_max: 2499.61 2499.63
No_80_layers
depth_min,depth_max: 2504.1 2504.14
No_90_layers
depth_min,depth_max: 2515.75 2515.9
No_100_layers
depth_min,depth_max: 2521.0 2521.1
No_110_layers
depth_min,depth_max: 2532.36 2532.56
No_120_layers
depth_min,depth_max: 2538.32 2539.92
No_130_layers
depth_min,depth_max: 2547.133 2549.21
No_140_layers
depth_min,depth_max: 2560.81 2564.26
No_150_layers
depth_min,depth_max: 2585.02 2586.58
No_160_layers
Label Work Finished!


In [28]:
wellname_colunm_data = pd.DataFrame(pd_well_no, columns=["Well Name"])

In [29]:
facies_colunm_data = pd.DataFrame(facies_list, columns=["Facies"])

In [30]:
facies_colunm_data

,Facies
0,3
1,3
2,3
3,3
4,3
...,...
3956,no_label
3957,no_label
3958,no_label
3959,no_label


In [31]:
wellname_colunm_data

,Well Name
0,1
1,1
2,1
3,1
4,1
...,...
3956,1
3957,1
3958,1
3959,1


In [32]:
pd_data = pd.concat([input_vector_data, wellname_colunm_data,facies_colunm_data],axis=1)        

In [33]:
pd_data

,DEPTH,GR,CNL,DT,DEN,MSFL,RS,RD,Well Name,Facies
0,2350.0858,113.4619,15.9331,81.0,2.5902,11.5818,6.9659,7.4271,1,3
1,2350.1858,116.5655,16.5907,82.8,2.6165,7.7921,7.0260,7.5060,1,3
2,2350.2858,120.4164,17.4862,85.6,2.6315,6.2955,6.8697,7.3354,1,3
3,2350.3858,124.4936,18.4462,88.6,2.6291,6.0488,6.5242,6.9516,1,3
4,2350.4858,128.6494,19.1042,91.0,2.6026,5.9448,6.1002,6.5036,1,3
...,...,...,...,...,...,...,...,...,...,...
3956,2745.6858,89.4620,7.3066,71.0,2.5733,7.0714,12.5438,13.1089,1,no_label
3957,2745.7858,95.4042,7.5931,71.0,2.5846,6.9656,12.6673,13.2187,1,no_label
3958,2745.8858,104.3638,8.0731,71.0,2.5926,6.8015,12.8980,13.4263,1,no_label
3959,2745.9858,113.7063,8.9307,71.0,2.5983,6.5375,13.2500,13.7623,1,no_label


In [34]:
data_save = pd_data[pd_data["Facies"] != "no_label"].reset_index(drop=True)

In [35]:
data_save

,DEPTH,GR,CNL,DT,DEN,MSFL,RS,RD,Well Name,Facies
0,2350.0858,113.4619,15.9331,81.0,2.5902,11.5818,6.9659,7.4271,1,3
1,2350.1858,116.5655,16.5907,82.8,2.6165,7.7921,7.0260,7.5060,1,3
2,2350.2858,120.4164,17.4862,85.6,2.6315,6.2955,6.8697,7.3354,1,3
3,2350.3858,124.4936,18.4462,88.6,2.6291,6.0488,6.5242,6.9516,1,3
4,2350.4858,128.6494,19.1042,91.0,2.6026,5.9448,6.1002,6.5036,1,3
...,...,...,...,...,...,...,...,...,...,...
2360,2586.0858,140.2403,17.6334,93.8,2.6152,5.6501,5.4704,4.4547,1,2
2361,2586.1858,139.1306,17.2158,91.2,2.6213,7.0352,5.6954,4.7051,1,2
2362,2586.2858,136.8668,16.5758,89.2,2.6312,8.4178,5.9588,4.9788,1,2
2363,2586.3858,134.5348,15.9358,88.0,2.6424,8.8941,6.1904,5.2095,1,2


In [36]:
# facies_id_list = np.full(len(data_save),0)

In [37]:
# for i in range(len(data_save)):
#     current_facies_id = facies_labels.index(data_save["Facies"][i])
#     # print(current_facies_id)

In [38]:
# facies_id = pd.DataFrame(current_facies_id, columns=["Facies_id"])

In [39]:
# pd_data = pd.concat([pd_data,facies_id],axis=1)       

## Label File Results Save（标记结果保存）

### Result files header information（结果文件文件头信息）

In [ ]:
well_name = filename_facies_logs.split(".csv")[0]
print(well_name)

In [ ]:
# filename_A ： 'HP1_orginLog_6D_4075m-4280m_R_0.125.csv'
well_name

In [42]:
data_save

,DEPTH,GR,CNL,DT,DEN,MSFL,RS,RD,Well Name,Facies
0,2350.0858,113.4619,15.9331,81.0,2.5902,11.5818,6.9659,7.4271,1,3
1,2350.1858,116.5655,16.5907,82.8,2.6165,7.7921,7.0260,7.5060,1,3
2,2350.2858,120.4164,17.4862,85.6,2.6315,6.2955,6.8697,7.3354,1,3
3,2350.3858,124.4936,18.4462,88.6,2.6291,6.0488,6.5242,6.9516,1,3
4,2350.4858,128.6494,19.1042,91.0,2.6026,5.9448,6.1002,6.5036,1,3
...,...,...,...,...,...,...,...,...,...,...
2360,2586.0858,140.2403,17.6334,93.8,2.6152,5.6501,5.4704,4.4547,1,2
2361,2586.1858,139.1306,17.2158,91.2,2.6213,7.0352,5.6954,4.7051,1,2
2362,2586.2858,136.8668,16.5758,89.2,2.6312,8.4178,5.9588,4.9788,1,2
2363,2586.3858,134.5348,15.9358,88.0,2.6424,8.8941,6.1904,5.2095,1,2


In [43]:
depth_info = data_save[depth_cols]

In [44]:
depth_info

0       2350.0858
1       2350.1858
2       2350.2858
3       2350.3858
4       2350.4858
          ...    
2360    2586.0858
2361    2586.1858
2362    2586.2858
2363    2586.3858
2364    2586.4858
Name: DEPTH, Length: 2365, dtype: float64

In [45]:
begin_depth = depth_info[0]
end_depth = depth_info[len(depth_info)-1]

In [46]:
print("begin_depth,end_depth:",begin_depth,end_depth)

begin_depth,end_depth: 2350.0858 2586.4858


In [47]:
# fianl_facies_logs = well_name + '_'+ str(begin_depth) + '-'+ str(end_depth) +'_facies_vectors_' + str(1/resolution)+ '.csv' 
fianl_facies_logs = well_name + '_facies_vectors_' + str(1/resolution)+ '.csv' 

In [ ]:
print(fianl_facies_logs)

### Result dataset save location(结果数据集保存位置)

In [49]:
Using_DataPath = 'facies_data/train/'

In [50]:
# Using_DataPath = os.path.join(Using_DataPath,fianl_facies_logs)

In [51]:
resolution_ratio = 1 / resolution
resolution_ratio

0.01

### The saved result curve(保存的结果曲线)

In [52]:
data_save

,DEPTH,GR,CNL,DT,DEN,MSFL,RS,RD,Well Name,Facies
0,2350.0858,113.4619,15.9331,81.0,2.5902,11.5818,6.9659,7.4271,1,3
1,2350.1858,116.5655,16.5907,82.8,2.6165,7.7921,7.0260,7.5060,1,3
2,2350.2858,120.4164,17.4862,85.6,2.6315,6.2955,6.8697,7.3354,1,3
3,2350.3858,124.4936,18.4462,88.6,2.6291,6.0488,6.5242,6.9516,1,3
4,2350.4858,128.6494,19.1042,91.0,2.6026,5.9448,6.1002,6.5036,1,3
...,...,...,...,...,...,...,...,...,...,...
2360,2586.0858,140.2403,17.6334,93.8,2.6152,5.6501,5.4704,4.4547,1,2
2361,2586.1858,139.1306,17.2158,91.2,2.6213,7.0352,5.6954,4.7051,1,2
2362,2586.2858,136.8668,16.5758,89.2,2.6312,8.4178,5.9588,4.9788,1,2
2363,2586.3858,134.5348,15.9358,88.0,2.6424,8.8941,6.1904,5.2095,1,2


In [53]:
# 输出txt
# data_save.to_csv(os.path.join(Using_DataPath,fianl_facies_logs),mode='w',float_format='%.4f',sep='\t',index=None,header=True)
data_save.to_csv(os.path.join(Using_DataPath,fianl_facies_logs),mode='w',float_format='%.4f',encoding='GBK',index=None,header=True)
print("Algorithm is Finished!!")

Algorithm is Finished!!


In [ ]:
print(os.path.join(Using_DataPath,fianl_facies_logs))